# Магические методы (часть 1)
## __str__, __repr__, __eq__, __hash__, __len__

Магические методы (dunder methods) — специальные методы в Python, которые начинаются и заканчиваются двумя подчёркиваниями. Они позволяют определять поведение объектов в различных ситуациях: вывод на экран, сравнение, использование в коллекциях и т.д.

In [ ]:
# Простейший пример магического метода __str__
class SimpleClass:
    def __str__(self):
        return "Это моя строка"

obj = SimpleClass()
print(obj)  # Выведет: "Это моя строка"
print(str(obj))  # Тоже использует __str__

## __str__ vs __repr__: в чем разница?

**`__str__`** — для пользователей:
- Используется функциями `str()`, `print()`, f-строками
- Должен возвращать читаемое, "человеческое" представление

**`__repr__`** — для разработчиков:
- Используется функцией `repr()`, отладчиками, в интерактивном режиме
- Должен возвращать однозначное представление, по которому можно воссоздать объект
- Если `__str__` не определён, используется `__repr__`

In [ ]:
# Пример с книгой: демонстрация разницы между __str__ и __repr__
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
    
    def __str__(self):
        """Для пользователей (читаемое представление)"""
        return f'Книга "{self.title}" - {self.author}'
    
    def __repr__(self):
        """Для разработчиков (однозначное представление)"""
        return f'Book("{self.title}", "{self.author}")'

# Демонстрация
book = Book("Преступление и наказание", "Достоевский")

print("str(book):", str(book))     # Книга "Преступление и наказание" - Достоевский
print("repr(book):", repr(book))   # Book("Преступление и наказание", "Достоевский")
print("Просто book:", book)        # Используется __str__

## __eq__ — оператор сравнения (равенство)

Метод `__eq__` определяет поведение оператора `==`. Он должен:
1. Принимать два аргумента: `self` и `other`
2. Возвращать `True` если объекты равны, `False` — если нет
3. Обычно проверяет равенство по значению, а не по идентичности (`is`)

Также существует метод `__ne__` для оператора `!=`, но если он не определён, Python использует `not (self == other)`.

In [ ]:
# Пример с вектором: сравнение объектов
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        """Определяет поведение оператора =="""
        if not isinstance(other, Vector):
            return False
        return self.x == other.x and self.y == other.y
    
    def __ne__(self, other):
        """Определяет поведение оператора !="""
        return not self.__eq__(other)

# Демонстрация
v1 = Vector(1, 2)
v2 = Vector(1, 2)
v3 = Vector(3, 4)

print(f"v1 == v2: {v1 == v2}")  # True - одинаковые координаты
print(f"v1 == v3: {v1 == v3}")  # False - разные координаты
print(f"v1 != v3: {v1 != v3}")  # True - использует __ne__

## __hash__ — хеширование объектов

Метод `__hash__` позволяет объектам быть элементами множеств (`set`) и ключами словарей (`dict`).

**Важное правило:** если два объекта равны (`a == b`), то их хеши должны быть равны (`hash(a) == hash(b)`). Обратное не обязательно.

Если переопределить `__eq__`, то `__hash__` автоматически становится `None`, и объект становится нехешируемым, если не переопределить `__hash__`.

In [ ]:
# Пример со студентом: использование объектов как ключей словаря
class Student:
    def __init__(self, name, student_id):
        self.name = name
        self.id = student_id
    
    def __eq__(self, other):
        return self.id == other.id
    
    def __hash__(self):
        """Хэш должен быть одинаковым для равных объектов"""
        return hash(self.id)

# Демонстрация
s1 = Student("Анна", 123)
s2 = Student("Анна", 123)  # Тот же ID
s3 = Student("Иван", 456)  # Другой ID

print(f"hash(s1): {hash(s1)}")
print(f"hash(s2): {hash(s2)}")  # Одинаковый с s1, т.к. ID одинаковый
print(f"hash(s3): {hash(s3)}")

# Использование как ключей в словаре
students_dict = {s1: "отличник"}
print(f"s2 в словаре: {s2 in students_dict}")  # True, т.к. s1 == s2

## __len__ — получение длины объекта

Метод `__len__` позволяет использовать встроенную функцию `len()` с вашими объектами. Он должен возвращать неотрицательное целое число.

Интересный факт: метод `__bool__`, который определяет истинность объекта в условных выражениях, по умолчанию вызывает `__len__` (если он определён) и считает объект `False`, если длина равна 0.

In [ ]:
# Пример с плейлистом: объект-коллекция
class Playlist:
    def __init__(self):
        self.songs = []
    
    def add_song(self, song):
        self.songs.append(song)
    
    def __len__(self):
        """Возвращает количество элементов"""
        return len(self.songs)
    
    def __bool__(self):
        """Определяет поведение в булевом контексте"""
        return len(self) > 0

# Демонстрация
my_playlist = Playlist()
print(f"Начальная длина: {len(my_playlist)}")  # 0

my_playlist.add_song("Bohemian Rhapsody")
my_playlist.add_song("Stairway to Heaven")
print(f"После добавления: {len(my_playlist)}")  # 2

# Использование в условном выражении
if my_playlist:  # Вызывает __bool__, который использует __len__
    print("Плейлист не пустой")
else:
    print("Плейлист пустой")

## Комплексный пример: класс Fraction (дробь)

Давайте создадим класс для работы с дробями, который будет использовать все изученные магические методы. Это поможет увидеть, как они работают вместе.

In [ ]:
class Fraction:
    """Класс для работы с дробями"""
    def __init__(self, numerator, denominator):
        self.numer = numerator
        self.denom = denominator
    
    def __str__(self):
        return f"{self.numer}/{self.denom}"
    
    def __repr__(self):
        return f"Fraction({self.numer}, {self.denom})"
    
    def __eq__(self, other):
        if not isinstance(other, Fraction):
            return False
        # Приводим к общему знаменателю: a/b == c/d если a*d == c*b
        return self.numer * other.denom == other.numer * self.denom
    
    def __hash__(self):
        # Хешируем кортеж из числителя и знаменателя
        return hash((self.numer, self.denom))
    
    def __len__(self):
        # Нестандартное использование: возвращаем сумму модулей
        return abs(self.numer) + abs(self.denom)

# Создаём дроби
f1 = Fraction(1, 2)
f2 = Fraction(2, 4)  # Эквивалентно 1/2
f3 = Fraction(3, 4)  # Другая дробь

print(f"str: {f1}")
print(f"repr: {repr(f1)}")
print(f"f1 == f2: {f1 == f2}")  # True, т.к. 1/2 == 2/4
print(f"f1 == f3: {f1 == f3}")  # False
print(f"hash(f1): {hash(f1)}")
print(f"len(f1): {len(f1)}")  # |1| + |2| = 3

# Использование в коллекциях
fractions_set = {f1, f2, f3}
print(f"Уникальных дробей в множестве: {len(fractions_set)}")  # 2 (f1 и f3 равны по __eq__)

## Важные правила и предостережения

1. **Согласованность `__eq__` и `__hash__`**: Если `a == b`, то обязательно `hash(a) == hash(b)`. Обратное не требуется.

2. **Неизменяемость**: Если объект изменяемый, его хеш не должен меняться. Лучше не делать изменяемые объекты хешируемыми.

3. **Возвращаемые значения**: `__len__` должен возвращать неотрицательное целое, `__hash__` — целое число.

4. **Производительность**: `__hash__` должен быть быстрым, так как вызывается часто при работе со словарями и множествами.

In [ ]:
# Демонстрация проблемы с изменяемыми объектами
class BadMutable:
    def __init__(self, value):
        self.value = value
    
    def __eq__(self, other):
        return self.value == other.value
    
    def __hash__(self):
        return hash(self.value)
    
    def change(self, new_value):
        self.value = new_value

# Проблема: объект может измениться после добавления в множество
obj = BadMutable(5)
my_set = {obj}
print(f"Объект в множестве: {obj in my_set}")  # True

obj.change(10)  # Меняем значение
print(f"Хеш изменился: {hash(obj)}")
print(f"Объект всё ещё в множестве? {obj in my_set}")  # Может быть False или ошибка

# Правильный подход: использовать только неизменяемые атрибуты для хеширования
class GoodImmutable:
    def __init__(self, value):
        self._value = value  # Защищённый атрибут
    
    @property
    def value(self):
        return self._value
    
    def __eq__(self, other):
        return self._value == other._value
    
    def __hash__(self):
        return hash(self._value)

## Практические советы по использованию

### Когда использовать каждый метод:
- **`__str__`** — когда нужен красивый вывод для пользователя (в интерфейсе, логах)
- **`__repr__`** — для отладки, должен позволять воссоздать объект
- **`__eq__`** — когда нужно сравнение по значению (а не по ссылке)
- **`__hash__`** — когда объекты будут ключами словаря или элементами множества
- **`__len__`** — для объектов-коллекций (списки, словари, множества)

### Рекомендации:
1. Всегда определяйте `__repr__` — это помогает при отладке
2. Если определили `__eq__`, определите и `__hash__` (или явно установите `__hash__ = None` для изменяемых объектов)
3. `__len__` должен быть быстрым — не выполняйте сложные вычисления

In [ ]:
# Пример класса с хорошей реализацией всех методов
class SmartCollection:
    def __init__(self, name, items=None):
        self.name = name
        self._items = items or []
    
    def __repr__(self):
        return f"SmartCollection('{self.name}', {self._items})"
    
    def __str__(self):
        return f"Коллекция '{self.name}' с {len(self)} элементами"
    
    def __eq__(self, other):
        return (self.name == other.name and 
                self._items == other._items)
    
    def __hash__(self):
        # Хешируем только name, т.к. items может изменяться
        return hash(self.name)
    
    def __len__(self):
        return len(self._items)
    
    def add_item(self, item):
        self._items.append(item)

# Демонстрация
coll1 = SmartCollection("numbers", [1, 2, 3])
coll2 = SmartCollection("numbers", [1, 2, 3])
coll3 = SmartCollection("letters", ['a', 'b'])

print(repr(coll1))
print(str(coll1))
print(f"coll1 == coll2: {coll1 == coll2}")
print(f"coll1 == coll3: {coll1 == coll3}")
print(f"Длина coll1: {len(coll1)}")
print(f"Хеш coll1: {hash(coll1)}")

# Использование в словаре (только name используется для хеширования)
collections_dict = {coll1: "первая коллекция"}
print(f"coll2 в словаре: {coll2 in collections_dict}")  # True, т.к. name одинаковый

## Заключение

Магические методы — мощный инструмент Python, который делает ваши классы:
1. **Интуитивно понятными** — работают со стандартными функциями и операторами
2. **Удобными в использовании** — интеграция со встроенными типами и функциями
3. **Эффективными** — могут использоваться в высокооптимизированных структурах данных

**Запомните ключевые моменты:**
- `__str__` для людей, `__repr__` для разработчиков
- `__eq__` и `__hash__` должны быть согласованы
- `__len__` определяет длину, а также влияет на `bool()` по умолчанию

Попробуйте реализовать эти методы в своих классах — это значительно улучшит их удобство и читаемость!